In [1]:
import numpy as np
import scipy as sp
import suite2p as s2p
import TwoPUtils
import os
from matplotlib import pyplot as plt

%matplotlib inline

%load_ext autoreload
%autoreload 2

In [9]:
source_folder  = os.path.join('/media','mplitt','Backup Plus1','2P_Data')
write_folder = os.path.join('/mnt','BigDisk','2P_scratch')


mouse = '4467331.1'

file_list = ({'date': '29_11_2020', 'scene':'YMaze_LNovel','session':1,'scan':3},
             {'date': '30_11_2020', 'scene':'YMaze_LNovel','session':1,'scan':7},
             {'date': '01_12_2020', 'scene':'YMaze_LNovel','session':1,'scan':4},
             {'date': '02_12_2020', 'scene':'YMaze_LNovel','session':1,'scan':3},
             {'date': '03_12_2020', 'scene':'YMaze_LNovel','session':1,'scan':8},
             {'date': '04_12_2020', 'scene':'YMaze_LNovel','session':1,'scan':2},
             {'date': '05_12_2020', 'scene':'YMaze_LNovel','session':1,'scan':2}, 
             {'date':'05_12_2020','scene':'YMaze_RewardReversal', 'session':1,'scan':4},
             {'date': '06_12_2020', 'scene':'YMaze_RewardReversal','session':2,'scan':1},
             {'date': '07_12_2020', 'scene':'YMaze_RewardReversal','session':1,'scan':3}, 
             {'date':'07_12_2020','scene':'YMaze_LNovel', 'session':1,'scan':5})



In [10]:
def check_and_make_h5s(file_list):
    h5list, framerates = [], []
    for f in file_list:
        print(f)
        scan_str = "%s_%03d_%03d" % (f['scene'],f['session'],f['scan'])

        source_stem = os.path.join(source_folder,mouse,f['date'],f['scene'],scan_str)
        file_stem = os.path.join(write_folder,mouse,f['date'],f['scene'],scan_str)
        info = TwoPUtils.scanner_tools.sbx_utils.loadmat(source_stem+'.mat')
        h5path = os.path.join(file_stem,scan_str+".h5")
#         if not os.path.exists(h5path):
#             _ = TwoPUtils.scanner_tools.sbx_utils.sbx2h5(source_stem,output_name=h5path,force_2chan=True)

        h5list.append(file_stem)
        framerates.append(info['frame_rate'])
    print(h5list)
    return h5list, framerates

In [11]:
f= file_list[0]
scan_str = "%s_%03d_%03d" % (f['scene'],f['session'],f['scan'])
source_stem = os.path.join(source_folder,mouse,f['date'],f['scene'],scan_str)
file_stem = os.path.join(write_folder,mouse,f['date'],f['scene'],scan_str)
info = TwoPUtils.scanner_tools.sbx_utils.loadmat(source_stem+'.mat')
for k,v in info.items():
    print(k,v)

frame [  133   133   133 ... 34307 34307 34310]
line [135 330 492 ... 238 399  50]
event_id [1 1 1 ... 1 1 1]
resfreq 7916
postTriggerSamples 5000
recordsPerBuffer 512
bytesPerBuffer 10240000
channels 1
ballmotion []
abort_bit 0
scanbox_version 2
scanmode 1
config {'wavelength': 980, 'frames': 0, 'lines': 512, 'magnification': 3, 'magnification_list': array(['1.0', '1.2', '1.4', '1.7', '2.0', '2.4', '2.8', '3.4', '4.0',
       '4.8', '5.7', '6.7', '8.0'], dtype='<U3'), 'pmt0_gain': 0.75, 'pmt1_gain': 0.59, 'knobby': {'pos': {'x': -41.51, 'y': 70.34, 'z': 0.78, 'a': 0}, 'schedule': array([[  0,   0,  10,   0,  30],
       [  0,   0,  10,   0,  60],
       [  0,   0,  10,   0,  90],
       [  0,   0,  10,   0, 120],
       [  0,   0,  10,   0, 150],
       [  0,   0,  10,   0, 180]], dtype=uint8)}}
sz [512 796]
fold_lines 0
otwave []
otwave_um []
otparam []
otwavestyle 1
volscan 0
power_depth_link 0
opto2pow []
area_line 1
calibration [<scipy.io.matlab.mio5_params.mat_struct object at 0x

In [5]:
ops = np.load(os.path.join(write_folder,mouse,"YMazeAll","suite2p","plane0","ops.npy"),allow_pickle=True).all()
ops['save_path0'] = os.path.join(write_folder,mouse,"YMazeAll")
ops['save_path'] = os.path.join(write_folder,mouse,"YMazeAll","suite2p","plane0")
ops['ops_path'] = os.path.join(write_folder,mouse,"YMazeAll","suite2p","plane0","ops.npy")
ops['reg_file'] = os.path.join(write_folder,mouse,"YMazeAll","suite2p","plane0","data.bin")
ops['reg_file_chan2'] = os.path.join(write_folder,mouse,"YMazeAll","suite2p","plane0","data_chan2.bin")
np.save(ops['ops_path'],ops)

In [7]:
h5list_all, frs_all = check_and_make_h5s(file_list)
ops = TwoPUtils.s2p.set_ops(d={'data_path': h5list_all,
                               'save_path0': os.path.join(write_folder,mouse,"YMazeAll"),
                                'fast_disk':[],
                                'move_bin':True,
                                'two_step_registration':True,
                                'maxregshiftNR':10,
                                'nchannels':2,
                                'tau':.7,
                                'functional_chan':1,
                                'fs':frs_all[0],
                                'roidetect':True,
                                'input_format':"h5",
                                'h5py_key':'data',
                                'nchannels':2,
                                'saveNWB':True,
                                'threshold_scaling':.75,
                                'chan2_thresh':.01})
ops=s2p.run_s2p(ops=ops)

{'date': '29_11_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 3}
{'date': '30_11_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 7}
{'date': '01_12_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 4}
{'date': '02_12_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 3}
{'date': '03_12_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 8}
{'date': '04_12_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 2}
{'date': '05_12_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 2}
{'date': '05_12_2020', 'scene': 'YMaze_RewardReversal', 'session': 1, 'scan': 4}
{'date': '06_12_2020', 'scene': 'YMaze_RewardReversal', 'session': 2, 'scan': 1}
{'date': '07_12_2020', 'scene': 'YMaze_RewardReversal', 'session': 1, 'scan': 3}
{'date': '07_12_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 5}
['/mnt/BigDisk/2P_scratch/4467331.1/29_11_2020/YMaze_LNovel/YMaze_LNovel_001_003', '/mnt/BigDisk/2P_scratch/4467331.1/30_11_2020/YMaze_LNovel/YMaze_LNovel_001_007', '/mnt/Bi

In [14]:
_file_list = ({'date': '29_11_2020', 'scene':'YMaze_LNovel','session':1,'scan':3},
             {'date': '30_11_2020', 'scene':'YMaze_LNovel','session':1,'scan':7},
             {'date': '01_12_2020', 'scene':'YMaze_LNovel','session':1,'scan':4},
             {'date': '02_12_2020', 'scene':'YMaze_LNovel','session':1,'scan':3},
             {'date': '03_12_2020', 'scene':'YMaze_LNovel','session':1,'scan':8},
             {'date': '04_12_2020', 'scene':'YMaze_LNovel','session':1,'scan':2},
            {'date': '06_12_2020', 'scene':'YMaze_RewardReversal','session':2,'scan':1})
for f in _file_list:
    _h5list,_frs = check_and_make_h5s([f,])
    ops = TwoPUtils.s2p.set_ops(d={'data_path': _h5list,
                                   'save_path0': _h5list[0],
                                    'fast_disk':[],
                                    'move_bin':True,
                                    'two_step_registration':True,
                                    'maxregshiftNR':10,
                                    'nchannels':2,
                                    'tau':.7,
                                    'functional_chan':1,
                                    'fs':_frs[0],
                                    'roidetect':True,
                                    'input_format':"h5",
                                    'h5py_key':'data',
                                    'nchannels':2})
    ops=s2p.run_s2p(ops=ops)

             

{'date': '29_11_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 3}
['/media/mplitt/BigDisk/2P_scratch/4467331.1/29_11_2020/YMaze_LNovel/YMaze_LNovel_001_003']
{}
h5
** Found 1 h5 files - converting to binary **
NOTE: using a list of h5 files:
['/media/mplitt/BigDisk/2P_scratch/4467331.1/29_11_2020/YMaze_LNovel/YMaze_LNovel_001_003/YMaze_LNovel_001_003.h5']
time 735.08 sec. Wrote 34310 frames per binary for 1 planes
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
NOTE: not registered / registration forced with ops['do_registration']>1
      (no previous offsets to delete)
----------- REGISTRATION
registering 34310 frames
NOTE: estimated bidiphase offset from data: 0 pixels
Reference frame, 17.67 sec.
Registered 2000/34310 in 76.31s
Registered 4000/34310 in 152.31s
Registered 6000/34310 in 229.51s
Registered 8000/34310 in 307.96s
Registered 10000/34310 in 395.75s
Registered 12000/34310 in 484.67s
Registered 14000/34310 in 572.20s
Registered 16000/34310 in 660.54s
Registered 18

In [15]:
_file_list = ({'date': '05_12_2020', 'scene':'YMaze_LNovel','session':1,'scan':2}, 
             {'date':'05_12_2020','scene':'YMaze_RewardReversal', 'session':1,'scan':4})
_h5list, _frs = check_and_make_h5s(_file_list)

ops = TwoPUtils.s2p.set_ops(d={'data_path': _h5list,
                               'save_path0': os.path.join(write_folder,mouse,"05_12_2020","combined"),
                                'fast_disk':[],
                                'move_bin':True,
                                'two_step_registration':True,
                                'maxregshiftNR':10,
                                'nchannels':2,
                                'tau':.7,
                                'functional_chan':1,
                                'fs':_frs[0],
                                'roidetect':True,
                                'input_format':"h5",
                                'h5py_key':'data',
                                'nchannels':2})
ops=s2p.run_s2p(ops=ops)


{'date': '05_12_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 2}
{'date': '05_12_2020', 'scene': 'YMaze_RewardReversal', 'session': 1, 'scan': 4}
['/media/mplitt/BigDisk/2P_scratch/4467331.1/05_12_2020/YMaze_LNovel/YMaze_LNovel_001_002', '/media/mplitt/BigDisk/2P_scratch/4467331.1/05_12_2020/YMaze_RewardReversal/YMaze_RewardReversal_001_004']
{}
h5
** Found 2 h5 files - converting to binary **
NOTE: using a list of h5 files:
['/media/mplitt/BigDisk/2P_scratch/4467331.1/05_12_2020/YMaze_LNovel/YMaze_LNovel_001_002/YMaze_LNovel_001_002.h5', '/media/mplitt/BigDisk/2P_scratch/4467331.1/05_12_2020/YMaze_RewardReversal/YMaze_RewardReversal_001_004/YMaze_RewardReversal_001_004.h5']
time 610.93 sec. Wrote 30771 frames per binary for 1 planes
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
NOTE: not registered / registration forced with ops['do_registration']>1
      (no previous offsets to delete)
----------- REGISTRATION
registering 30771 frames
NOTE: estimated bidiphase offset f

In [16]:
_file_list = ({'date': '07_12_2020', 'scene':'YMaze_RewardReversal','session':1,'scan':3}, 
             {'date':'07_12_2020','scene':'YMaze_LNovel', 'session':1,'scan':5})

_h5list,_frs = check_and_make_h5s(_file_list)

ops = TwoPUtils.s2p.set_ops(d={'data_path': _h5list,
                               'save_path0': os.path.join(write_folder,mouse,"07_12_2020","combined"),
                                'fast_disk':[],
                                'move_bin':True,
                                'two_step_registration':True,
                                'maxregshiftNR':10,
                                'nchannels':2,
                                'tau':.7,
                                'functional_chan':1,
                                'fs':_frs[0],
                                'roidetect':True,
                                'input_format':"h5",
                                'h5py_key':'data',
                                'nchannels':2})
ops=s2p.run_s2p(ops=ops)

{'date': '07_12_2020', 'scene': 'YMaze_RewardReversal', 'session': 1, 'scan': 3}
{'date': '07_12_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 5}
['/media/mplitt/BigDisk/2P_scratch/4467331.1/07_12_2020/YMaze_RewardReversal/YMaze_RewardReversal_001_003', '/media/mplitt/BigDisk/2P_scratch/4467331.1/07_12_2020/YMaze_LNovel/YMaze_LNovel_001_005']
{}
h5
** Found 2 h5 files - converting to binary **
NOTE: using a list of h5 files:
['/media/mplitt/BigDisk/2P_scratch/4467331.1/07_12_2020/YMaze_RewardReversal/YMaze_RewardReversal_001_003/YMaze_RewardReversal_001_003.h5', '/media/mplitt/BigDisk/2P_scratch/4467331.1/07_12_2020/YMaze_LNovel/YMaze_LNovel_001_005/YMaze_LNovel_001_005.h5']
time 632.21 sec. Wrote 30209 frames per binary for 1 planes
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
NOTE: not registered / registration forced with ops['do_registration']>1
      (no previous offsets to delete)
----------- REGISTRATION
registering 30209 frames
NOTE: estimated bidiphase offset f

In [18]:
for h5 in h5list_all:
    !rm {h5}/*.h5 

In [13]:
!rm {h5list[-1]}/*.h5
